### Task
Your goal is to solve 8-puzzle problems using different search techniques learned in the class
and compare them on their run time and number of nodes generated.
Specifically, code these 5 algorithms
1. BFS (Breath First Search)
2. IDS (Iterative deepening DFS) (you may want to check for cycles during DFS)
3. A* with misplaced title heuristic. (h1)
4. A* with Manhattan distance heuristic (h2).
5. A* with one more heuristic (invent or check the literature for this) (h3)

Note 1: Please feel free to use the code for the AIMA book at https://github.com/aimacode. \
Note 2: For a new 8-puzzle heuristic, either invent one yourself or find one from the literature
(ref. Exercises 3.39 and 3.40 at https://aimacode.github.io/aima-exercises/search-exercises/).

### Part 1: (40 pts)
Write a code that will scan the input file from a given path and solve it using one of the five
algorithms (you need to code all five algorithms).

### Input: Two command line arguments:
1. File path and
2. Algorithm to be used (BFS/IDS/h1/h2/h3).

### Output:
1. Total nodes generated (for A* this includes nodes in closed list and fringe).
2. Total time taken.
3. A valid sequence of actions that will take the given state to the goal state.
a. Please note that the meaning of action is important (action is associated with the
movement of a tile, not with the movement of the blank space). For example, in
Fig 1 above, the action sequence is DRUL.
4. Also note that not all puzzles are solvable (See Appendix A1). Your code should check whether the puzzle is solvable or not before attempting to solve it.

### A note on efficiency:

* Some of the algorithms may take a prohibitive amount of time. In such cases terminate
your code after a maximum of 15 minutes.
* A* algorithm should not run into memory or time issues if implemented properly. 

Below are sample outputs for A* for two heuristics h2 (Manhattan distance) and h1 (misplaced tiles). Your code should work in a similar fashion for other algorithms. Please note: 
* The exact format of the output is not important; it should capture all the information though.
* Your solution can take different paths and can generate a different number of nodes. Also, different heuristics could give different optimal paths. However, the optimal path length should be the same.

In [23]:
import sys
from collections import deque

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

In [20]:
file_path = "../Part2/S1.txt"

sample = open(file_path, 'r').read()
print(test)

3 5 2
6 1 7
_ 8 4



In [21]:
sample_arr = sample.split()
sample_arr

['3', '5', '2', '6', '1', '7', '_', '8', '4']

In [24]:
goal_state = ['1', '2', '3', '4', '5', '6', '7', '8', '_']

### Problem

In [25]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

## Node

In [26]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

### GraphProblem

In [27]:
class GraphProblem(Problem):

    """The problem of searching a graph from one node to another."""

    def __init__(self, initial, goal, graph):
        Problem.__init__(self, initial, goal)
        self.graph = graph

    def actions(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get(A).keys())

    def result(self, state, action):
        """The result of going to a neighbor is just that neighbor."""
        return action

    def path_cost(self, cost_so_far, A, action, B):
        return cost_so_far + (self.graph.get(A, B) or infinity)

    def find_min_edge(self):
        """Find minimum value of edges."""
        m = infinity
        for d in self.graph.graph_dict.values():
            local_min = min(d.values())
            m = min(m, local_min)

        return m

    def h(self, node):
        """h function is straight-line distance from a node's state to goal."""
        locs = getattr(self.graph, 'locations', None)
        if locs:
            if type(node) is str:
                return int(distance(locs[node], locs[self.goal]))

            return int(distance(locs[node.state], locs[self.goal]))
        else:
            return infinity

In [ ]:
# Heuristics for 8 Puzzle Problem
import math

def linear(node):
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])

def manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0
    
    for i in range(len(state)):
        index_state[state[i]] = index[i]
    
    mhd = 0
    
    for i in range(8):
        for j in range(2):
            mhd = abs(index_goal[i][j] - index_state[i][j]) + mhd
    
    return mhd

def sqrt_manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0
    
    for i in range(len(state)):
        index_state[state[i]] = index[i]
    
    mhd = 0
    
    for i in range(8):
        for j in range(2):
            mhd = (index_goal[i][j] - index_state[i][j])**2 + mhd
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

In [ ]:
# Solving the puzzle 
puzzle = EightPuzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle.check_solvability((2, 4, 3, 1, 5, 6, 7, 8, 0)) # checks whether the initialized configuration is solvable or not

In [ ]:
astar_search(puzzle).solution()

In [ ]:
astar_search(puzzle, manhattan).solution()

In [ ]:
astar_search(puzzle, sqrt_manhattan).solution()

In [ ]:
astar_search(puzzle, max_heuristic).solution()

And here's how recursive_best_first_search can be used to solve this problem too.

In [ ]:
recursive_best_first_search(puzzle, manhattan).solution()